# Programming with dependent types in Agda

- We need to be very generous with spaces. For example "x:" is a legal identifier in Agda.

## Simple types

In [21]:
-- lab00-bool.agda

module lab00-bool where

-- simple type
-- the type Bool is inhabited precisely by two elements
data Bool : Set where
    true : Bool
    false : Bool

-- we can define a computable functions on booleans
and : Bool → Bool → Bool
and true true = true
and _ _ = false

or : Bool → Bool → Bool
or true _ = true
or _ true = true
or false false = false

not : Bool → Bool
not true = false
not false = true

Checking lab00-bool (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-bool.agda).


## Inductive types

In [129]:
-- lab00-nat.agda

module lab00-nat where

open import lab00-bool

-- inductive type
-- natural numbers
data ℕ : Set where
  zero : ℕ
  suc  : ℕ → ℕ
  
-- non recursive function on an inductive type
pred : ℕ → ℕ
pred (suc n) = n
pred _ = zero -- must be total!

infixr 5 _+_

-- recursive function on an inductive type
_+_ : ℕ → ℕ → ℕ
zero + m = m
(suc n) + m = suc (n + m)

-- natural numbers have a decidable equality
natEq : ℕ → ℕ → Bool
natEq zero    zero    = true
natEq (suc n) (suc m) = natEq n m
natEq _ _ = false

-- we can use decimal numerals
{-# BUILTIN NATURAL ℕ #-}
n = zero + 2
m = 3 + 5 + 1

Checking lab00-nat (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-nat.agda).


**Exercise.** Define multiplication on natural numbers `_*_ : ℕ → ℕ → ℕ`.

In [31]:
-- lab00-nat-mul.agda

module lab00-nat-mul where

open import lab00-nat

infixr 6 _*_

_*_ : ℕ → ℕ → ℕ
zero * _  =  zero
suc m * n  =  n + (m * n)

Checking lab00-nat-mul (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-nat-mul.agda).


**Exercise.** We can give a more efficient representation of natural numbers based on their binary expansion (using the least significant digit first convention).
1. Define a function `inc : Bin → Bin` to increment a natural number in binary.
2. Define two functions `to : ℕ → Bin` and `from : Bin → ℕ` to convert back and forth between the two representations.

We will prove in a later tutorial that the two functions are inverses of each other.

In [123]:
-- lab00-nat-bin.agda

module lab00-nat-bin where

data Bin : Set where
  nil : Bin
  x0_ : Bin → Bin
  x1_ : Bin → Bin
  
-- example numbers
m = x0 x1 nil
n = x1 x1 x0 x0 nil

OK

In [131]:
-- lab00-nat-bin-sol.agda

module lab00-nat-bin-sol where

open import lab00-nat
open import lab00-nat-mul
open import lab00-nat-bin

inc : Bin → Bin
inc nil = x1 nil
inc (x0 b) = x1 b
inc (x1 b) = x0 (inc b)

to : ℕ → Bin
to zero = x0 nil
to (suc n) = inc (to n)

from : Bin → ℕ
from nil = zero
from (x0 b) = 2 * from b
from (x1 b) = 2 * from b + 1

Checking lab00-nat-bin-sol (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-nat-bin-sol.agda).


## Parametrised types (polymorphism)

In [10]:
-- lab00-list.agda

module lab00-list where

open import lab00-nat

infixr 5 _::_
infixr 5 _++_

-- parametrised type
ident : (A : Set) → A → A
ident A x = x

identNat : ℕ → ℕ
identNat = ident ℕ 

-- in many cases we can avoid writing the parameter explicitly
-- (Agda finds it automatically for us)
id : {A : Set} → A → A
id x = x

idNat : ℕ → ℕ
idNat = id

const : {A B : Set} → A → B → A
const x _ = x

-- parametrised inductive type:
-- the parameter does not change during the induction
data _* (A : Set) : Set where
  ε : A *
  _::_ : A → A * → A *

-- concatenation of lists
_++_ : {A : Set} → A * → A * → A *
ε ++ ys = ys
(x :: xs) ++ ys = x :: (xs ++ ys)

map : {A B : Set} → (A → B) → A * → B *
map _ ε = ε
map f (a :: as) = f a :: map f as

foldr : {A B : Set} → (A → B → B) → B → A * → B
foldr _⊗_ e ε         = e
foldr _⊗_ e (x :: xs) = x ⊗ foldr _⊗_ e xs

Checking lab00-list (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-list.agda).


**Exercise.** Define the function `length` mapping a list to its length, using only `map` and `foldr`.

In [12]:
-- lab00-length.agda

module lab00-length where

open import lab00-nat
open import lab00-list

length : {A : Set} → A * → ℕ
length xs = foldr _+_ zero (map (const 1) xs)

Checking lab00-length (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-length.agda).


# Indexed types (type families)
Indexed types are type families indexed by terms of another type.

In [39]:
-- lab00-even.agda

module lab00-even where

open import lab00-nat

-- indexed inductive type:
-- elements at a given index depend on previously constructed elements on other indices
-- the index is a natural number
data Even : ℕ → Set where
  even-zero : Even zero
  even-plus2 : {n : ℕ} → Even n → Even (suc (suc n))
  
-- the sum of two even numbers is even
plus-long : {m n : ℕ} → Even m → Even n → Even (m + n)
plus-long {zero} {n} even-zero evn = evn
plus-long {suc(suc m)} {n} (even-plus2 evm) evn = even-plus2 {m + n} (plus-long {m} {n} evm evn)

-- in compact form
-- (Agda can correctly infer the implicit arguments)
plus : {m n : ℕ} → Even m → Even n → Even (m + n)
plus even-zero evn = evn
plus (even-plus2 evm) evn = even-plus2 (plus evm evn)

Checking lab00-even (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-even.agda).


**Exercise.** Define multiplication for `Even`.

In [36]:
-- lab00-even-mul.agda

module lab00-even-mul where

open import lab00-nat
open import lab00-nat-mul
open import lab00-even

-- the product of two even numbers is even
mul : {m n : ℕ} → Even m → Even n → Even (m * n)
mul even-zero _ = even-zero
mul (even-plus2 evm) evn = plus evn (plus evn (mul evm evn))

Checking lab00-even-mul (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-even-mul.agda).


**Exercise.** Define even and odd numbers by a *mutual* inductive definition. Define addition on them, by a *mutual* recursive definition.

In [42]:
-- lab00-even-odd.agda

module lab00-even-odd where

open import lab00-nat

data Even : ℕ → Set
data Odd : ℕ → Set

data Even where
-- TODO

data Odd where
-- TODO

Checking lab00-even-odd (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-even-odd.agda).


In [54]:
-- lab00-even-odd.agda

module lab00-even-odd where

open import lab00-nat

data Even : ℕ → Set
data Odd : ℕ → Set

data Even where
  zero : Even zero
  plusE : {n : ℕ} → Odd n → Even (suc n)
  
data Odd where
  plusO : {n : ℕ} → Even n → Odd (suc n)

-- plus functions defined by mutual recursion
plusEE : {m n : ℕ} → Even m → Even n → Even (m + n)
plusEO : {m n : ℕ} → Even m → Odd n → Odd (m + n)
plusOE : {m n : ℕ} → Odd m → Even n → Odd (m + n)
plusOO : {m n : ℕ} → Odd m → Odd n → Even (m + n)

plusEE zero n = n
plusEE (plusE m) n = plusE (plusOE m n)

plusEO zero n = n
plusEO (plusE m) n = plusO (plusOO m n)

plusOE (plusO m) n = plusO (plusEE m n)

plusOO (plusO m) n = plusE (plusEO m n)

Checking lab00-even-odd (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-even-odd.agda).


In [96]:
-- lab00-vector.agda

module lab00-vector where

open import lab00-nat
open import lab00-even

infixr 5 _::_
infixr 5 _++_

-- indexed and parametrised type
-- the parameter is the type A and the index is the dimension n
data Vector (A : Set) : ℕ → Set where
  [] : Vector A zero
  _::_ : {n : ℕ} → A → Vector A n → Vector A (suc n)

_++_ : {A : Set} {m n : ℕ} → Vector A m → Vector A n → Vector A (m + n)
[] ++ ys = ys
(x :: xs) ++ ys = x :: (xs ++ ys)

map : {A B : Set} {n : ℕ} → (A → B) → Vector A n → Vector B n
map _ [] = []
map f (x :: xs) = f x :: map f xs

foldr : {A B : Set} {n : ℕ} → (A → B → B) → B → Vector A n → B
foldr _⊗_ e [] = e
foldr _⊗_ e (x :: xs) = x ⊗ foldr _⊗_ e xs

-- need to access the hidden parameter!
length : {A : Set} {n : ℕ} → Vector A n → ℕ
length {A} {n} _ = n

-- define the type of vectors of even length
data VectorEven (A : Set) : Set where
--  [] : VectorEven zero
--  _::_::_ : {n : ℕ} → A → A → VectorEven A n → VectorEven A (suc (suc n))
    vec : {n : ℕ} → Even n → Vector A n → VectorEven A

_:::_:::_ : {A : Set} → A → A → VectorEven A → VectorEven A
x ::: y ::: (vec evx xs) = vec (even-plus2 evx) (x :: y :: xs)

_+++_ : {A : Set} → VectorEven A → VectorEven A → VectorEven A
(vec even-zero []) +++ (vec evy ys) = vec evy ys
(vec (even-plus2 evx) (x :: y :: xs)) +++ (vec evy ys) = x ::: y ::: (vec evx xs +++ vec evy ys)

mapve : {A B : Set} → (A → B) → VectorEven A → VectorEven B
mapve _ (vec even-zero [])= vec even-zero []
mapve f (vec (even-plus2 evx) (x :: y :: xs)) = f x ::: f y ::: mapve f (vec evx xs)

Checking lab00-vector (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-vector.agda).


In [ ]:
-- lab00-list-vector.agda

module lab00-list-vector where

open import lab00-nat

data ListVector (A : Set) : Set where
  [] : ListVector
  _::_ : {n : ℕ} → A → Vector A n → Vector A (suc n)


## Inductive-recursive types

In [104]:
-- lab00-inductive-recursive.agda

module lab00-inductive-recursive where

open import lab00-nat
open import lab00-even

-- infixr 5 _++_

-- list of numbers of even sum

-- declaration
data EvenSumList : Set
sum : EvenSumList → ℕ

-- inductive definition, relying on sum
data EvenSumList where
  [] : EvenSumList
  cons : (n : ℕ) → (ns : EvenSumList) → (Even (n + sum ns)) → EvenSumList

-- recursive definition
sum [] = zero
sum (cons n ns _) = n + sum ns

-- append preserves evenness
append : EvenSumList → EvenSumList → EvenSumList
append [] ns = ns
append (cons m ms even) ns = cons m (append ms ns) _

Checking lab00-inductive-recursive (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-inductive-recursive.agda).
Unsolved metas at the following locations:
  /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-inductive-recursive.agda:28,52-53
